In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/drive/MyDrive/WLLI_MALAYALAM/Dataset/Final_mal_train.csv')
df_val=pd.read_csv('/content/drive/MyDrive/WLLI_MALAYALAM/Dataset/Final_mal_dev.csv')

In [ ]:
df_train.head()

,WORD,LABEL
0,hoo,MALAYALAM
1,mammokka,NAME
2,police,ENGLISH
3,vesham,MALAYALAM
4,aaha,MALAYALAM


In [ ]:
df_train=df_train.dropna()

In [ ]:
!pip install indic-transliteration
!pip install word2number
!pip install inflect
import pandas as pd
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
import inflect
from word2number import w2n


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5566 sha256=e1e1028f75961a7c896adb4f4baa6948c43cb47b4c8141369d7b1ecf09eca4bb
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [ ]:
p = inflect.engine()
def romanized_to_malayalam(romanized_text):
    malayalam_text = transliterate(romanized_text, sanscript.ITRANS, sanscript.MALAYALAM)
    return malayalam_text

def number_to_words(number_text):
    try:
        number_value = w2n.word_to_num(number_text)
        word_format = p.number_to_words(number_value)
        return word_format
    except ValueError:
        return number_text


df = pd.DataFrame(df_train)

df['WORD'] = df.apply(lambda row: romanized_to_malayalam(row['WORD']) if row['LABEL'] == 'MALAYALAM' else row['WORD'], axis=1)
df['WORD'] = df.apply(lambda row: number_to_words(row['WORD']) if row['LABEL'] == 'NUMBER' else row['WORD'], axis=1)

print(df)

                                        WORD      LABEL
0                                         ഹൂ  MALAYALAM
1                                   mammokka       NAME
2                                     police    ENGLISH
3                                      വേശമ്  MALAYALAM
4                                         ആഹ  MALAYALAM
...                                      ...        ...
47250                                ഇന്ഗനാഅ  MALAYALAM
47251                           കേല്ക്കുന്നേ  MALAYALAM
47252  mammoookkkaaaaaaaaaaaaaaaaaaaaaaaaaaa      MIXED
47253                        romanjification      MIXED
47254                                      .        SYM

[47247 rows x 2 columns]


In [ ]:
df_val['WORD'] = df_val.apply(lambda row: romanized_to_malayalam(row['WORD']) if row['LABEL'] == 'MALAYALAM' else row['WORD'], axis=1)
df_val['WORD'] = df_val.apply(lambda row: number_to_words(row['WORD']) if row['LABEL'] == 'NUMBER' else row['WORD'], axis=1)

In [ ]:
df_data['LABEL'].value_counts()

LABEL
MALAYALAM    21507
ENGLISH      10398
SYM           5255
OTHER         3898
NAME          3349
MIXED         1525
NUMBER        1073
PLACE          242
Name: count, dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
print(len(df_data))

47247


In [ ]:
import nltk.data
import numpy as np
import nltk

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_data['LABEL'])
y_test =label_encoder.transform(df_val['LABEL'])

In [ ]:
import pandas as pd
import pickle
import numpy as np

# Load the combined vector dictionary from the pickle file
with open('/content/drive/MyDrive/WLLI_MALAYALAM/Dataset/embeddings_combined.pkl', 'rb') as pickle_file:
    combined_data = pickle.load(pickle_file)

# Extract English and Malayalam embedding dictionaries
embedding_index_en = combined_data['embedding_index_en']
embedding_index_ml = combined_data['embedding_index_ml']

# Load your dataset
dataset_path = '/content/drive/MyDrive/WLLI_MALAYALAM/Dataset/fasttextnumtowordmaltonative.csv'
df = pd.read_csv(dataset_path)

# Function to get the vector for a word
def get_vector(word, label):
    if label == 'ENGLISH' and word in embedding_index_en:
        return embedding_index_en[word]
    elif label == 'MALAYALAM' and word in embedding_index_ml:
        return embedding_index_ml[word]
    else:
        return oov_embeddings(word)

# Function to handle out-of-vocabulary (OOV) embeddings for labels other than English and Malayalam
def oov_embeddings(word):
    vector = []
    try:
        for ch in word.lower():
            vector.append(np.random.uniform(-1, 1, 300))
    except KeyError:
        vector.append(np.random.uniform(-1, 1, 300))
    return np.mean(np.vstack(np.array(vector)), axis=0)

# Apply the function to each row in the dataset
df['Vector'] = df.apply(lambda row: get_vector(row['WORD'], row['LABEL']), axis=1)

# Save the new DataFrame to a pickle file
df.to_pickle('/content/drive/MyDrive/WLLI_MALAYALAM/Dataset/processed_dataset.pkl')



In [ ]:
df_val['Vector'] = df_val.apply(lambda row: get_vector(row['WORD'], row['LABEL']), axis=1)

In [ ]:
df['Vector']

0        [0.002, 0.0652, 0.2384, -0.0065, -0.076, 0.079...
1        [0.08301946462769122, 0.09093095825763345, -0....
2        [-0.0409, -0.0193, 0.0521, -0.0052, -0.0804, 0...
3        [-0.5252608529474981, -0.19874493256562023, -0...
4        [-0.0077, 0.1571, 0.0596, 0.0819, -0.1031, 0.0...
                               ...                        
34924    [0.16679120589365876, -0.0494912515682407, -0....
34925    [-0.2998075121228228, 0.026534329254382483, -0...
34926    [-0.038274585664977566, -0.0005151915757475132...
34927    [0.045111266198451586, 0.07184621329804602, 0....
34928    [-0.29981652565576833, -0.9630131361531038, 0....
Name: Vector, Length: 34929, dtype: object

In [ ]:
import pickle #dont run this
with open('/content/drive/MyDrive/WLLI_MALAYALAM/Dataset/processed_dataset1.pkl', 'wb') as f:  # open a text file
    pickle.dump(df['Vector'], f) # serialize the list

In [ ]:
import pickle
with open('/content/drive/MyDrive/WLLI_MALAYALAM/Dataset/processed_dataset1.pkl','rb') as f:

    train_loaded = pickle.load(f) # deserialize using load()

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(train_loaded,df.LABEL, test_size=0.3)

In [ ]:
X_train=df['Vector']
X_test=df_val['Vector']
y_train=df['LABEL']
y_test=df_val['LABEL']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [ ]:
MNB = GaussianNB()
MNB.fit(X_train.tolist(), y_train.tolist())
y_pred_MNB= MNB.predict(X_test.tolist())
print( classification_report(y_test, y_pred_MNB))

              precision    recall  f1-score   support

     ENGLISH       0.65      0.87      0.75      2311
   MALAYALAM       0.57      0.26      0.36      4613
       MIXED       0.12      0.38      0.18       346
        NAME       0.09      0.13      0.11       761
      NUMBER       0.04      0.03      0.03       273
       OTHER       0.16      0.27      0.20       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.90      1.00      0.94      1197

    accuracy                           0.47     10479
   macro avg       0.32      0.37      0.32     10479
weighted avg       0.52      0.47      0.46     10479



In [ ]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train.tolist(), y_train.tolist())
y_pred_svm = svm_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

     ENGLISH       0.69      0.83      0.75      2311
   MALAYALAM       0.55      0.92      0.69      4613
       MIXED       0.00      0.00      0.00       346
        NAME       0.00      0.00      0.00       761
      NUMBER       0.00      0.00      0.00       273
       OTHER       0.00      0.00      0.00       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.00      0.00      0.00      1197

    accuracy                           0.59     10479
   macro avg       0.16      0.22      0.18     10479
weighted avg       0.39      0.59      0.47     10479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.svm import LinearSVC
lsvc_model = LinearSVC(C=1.0, class_weight='balanced')
lsvc_model.fit(X_train.tolist(), y_train.tolist())
y_pred_lsvc = lsvc_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_lsvc))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

     ENGLISH       0.61      0.92      0.74      2311
   MALAYALAM       0.63      0.61      0.62      4613
       MIXED       0.04      0.03      0.03       346
        NAME       0.09      0.02      0.04       761
      NUMBER       0.04      0.05      0.04       273
       OTHER       0.12      0.03      0.05       930
       PLACE       0.00      0.06      0.01        48
         SYM       0.19      0.11      0.14      1197

    accuracy                           0.49     10479
   macro avg       0.22      0.23      0.21     10479
weighted avg       0.46      0.49      0.46     10479



In [ ]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train.tolist(), y_train.tolist())
y_pred_lr = lr_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

     ENGLISH       0.64      0.67      0.66      2311
   MALAYALAM       0.55      0.93      0.69      4613
       MIXED       0.00      0.00      0.00       346
        NAME       0.00      0.00      0.00       761
      NUMBER       0.00      0.00      0.00       273
       OTHER       0.03      0.00      0.00       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.69      0.10      0.17      1197

    accuracy                           0.57     10479
   macro avg       0.24      0.21      0.19     10479
weighted avg       0.46      0.57      0.47     10479



In [ ]:
RF_model = RandomForestClassifier()
RF_model.fit(X_train.tolist(), y_train.tolist())
y_pred_rf = RF_model.predict(X_test.tolist())
print( classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

     ENGLISH       1.00      0.95      0.97      2311
   MALAYALAM       0.66      0.99      0.79      4613
       MIXED       0.00      0.00      0.00       346
        NAME       0.00      0.00      0.00       761
      NUMBER       0.00      0.00      0.00       273
       OTHER       0.20      0.00      0.00       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.91      1.00      0.95      1197

    accuracy                           0.76     10479
   macro avg       0.35      0.37      0.34     10479
weighted avg       0.63      0.76      0.67     10479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
DTmodel = DecisionTreeClassifier()
DTmodel.fit(X_train.tolist(), y_train.tolist())
y_predict_dt= DTmodel.predict(X_test.tolist())
print( classification_report(y_test,y_predict_dt))

              precision    recall  f1-score   support

     ENGLISH       0.92      0.94      0.93      2311
   MALAYALAM       0.67      0.66      0.66      4613
       MIXED       0.06      0.06      0.06       346
        NAME       0.14      0.16      0.15       761
      NUMBER       0.04      0.05      0.05       273
       OTHER       0.12      0.12      0.12       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.77      0.66      0.71      1197

    accuracy                           0.60     10479
   macro avg       0.34      0.33      0.34     10479
weighted avg       0.61      0.60      0.60     10479



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train.tolist(), y_train.tolist())
y_pred_knn = classifier.predict(X_test.tolist())
print( classification_report(y_test,y_pred_knn))

              precision    recall  f1-score   support

     ENGLISH       0.39      0.95      0.55      2311
   MALAYALAM       0.60      0.62      0.61      4613
       MIXED       0.00      0.00      0.00       346
        NAME       0.00      0.00      0.00       761
      NUMBER       0.00      0.00      0.00       273
       OTHER       0.04      0.00      0.00       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.00      0.00      0.00      1197

    accuracy                           0.48     10479
   macro avg       0.13      0.20      0.15     10479
weighted avg       0.35      0.48      0.39     10479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
voting = VotingClassifier(
    estimators=[('MNB', MNB),('RF', RF_model),(' DecisionTreeClassifier', DTmodel)],
    voting='hard')

In [ ]:
voting.fit(X_train.tolist(), y_train.tolist())

VotingClassifier(estimators=[('MNB', GaussianNB()),
                             ('RF', RandomForestClassifier()),
                             (' DecisionTreeClassifier',
                              DecisionTreeClassifier())])

In [ ]:
voting_pred= voting.predict(X_test.tolist())
print(classification_report(y_test, voting_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     ENGLISH       0.94      0.95      0.95      2311
   MALAYALAM       0.66      0.91      0.76      4613
       MIXED       0.17      0.02      0.04       346
        NAME       0.11      0.02      0.04       761
      NUMBER       0.00      0.00      0.00       273
       OTHER       0.13      0.04      0.06       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.90      1.00      0.95      1197

    accuracy                           0.73     10479
   macro avg       0.36      0.37      0.35     10479
weighted avg       0.63      0.73      0.66     10479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.2 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Initialize the CatBoost classifier
model = CatBoostClassifier(iterations=1000,  # Number of trees (default is 1000)
                           learning_rate=0.1,  # Learning rate (default is 0.03)
                           depth=6,  # Depth of the trees (default is 6)
                           loss_function='MultiClass',  # Loss function (default is 'Logloss')
                           verbose=100  # Controls the level of verbosity (default is 100)
                           )

# Train the model
model.fit(X_train.tolist(), y_train.tolist(), eval_set=(X_test.tolist(), y_test), early_stopping_rounds=50, verbose=100)

# Predictions on the test set
y_pred = model.predict(X_test.tolist())

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

0:	learn: 1.8199260	test: 1.8304334	best: 1.8304334 (0)	total: 2.58s	remaining: 42m 57s
100:	learn: 0.7354347	test: 0.8162747	best: 0.8162747 (100)	total: 1m 37s	remaining: 14m 26s
200:	learn: 0.6121256	test: 0.7901899	best: 0.7901899 (200)	total: 2m 55s	remaining: 11m 39s
300:	learn: 0.5172272	test: 0.7862465	best: 0.7861665 (296)	total: 4m 14s	remaining: 9m 52s
400:	learn: 0.4365014	test: 0.7848215	best: 0.7846767 (389)	total: 5m 32s	remaining: 8m 16s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.784676714
bestIteration = 389

Shrink model to first 390 iterations.
Accuracy: 0.7620001908579063


In [ ]:
print("\n", classification_report(y_test, y_pred))


               precision    recall  f1-score   support

     ENGLISH       0.99      0.96      0.97      2311
   MALAYALAM       0.66      0.99      0.79      4613
       MIXED       0.00      0.00      0.00       346
        NAME       0.00      0.00      0.00       761
      NUMBER       0.00      0.00      0.00       273
       OTHER       0.38      0.01      0.02       930
       PLACE       0.00      0.00      0.00        48
         SYM       0.94      1.00      0.97      1197

    accuracy                           0.76     10479
   macro avg       0.37      0.37      0.34     10479
weighted avg       0.65      0.76      0.67     10479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
y_train= label_encoder.fit_transform(y_train)

#y_train.unique()

In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
y_test= label_encoder.fit_transform(y_test)

#y_test.unique()

In [ ]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgbmodel.fit(X_train.tolist(), y_train.tolist())
val_pred2=xgbmodel.predict(X_test.tolist())
accuracy = accuracy_score(y_test, val_pred2)

print("Dev Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, val_pred2))

Dev Accuracy: 76.1809

               precision    recall  f1-score   support

           0       1.00      0.96      0.98      2311
           1       0.66      0.99      0.79      4613
           2       0.33      0.00      0.01       346
           3       0.16      0.01      0.02       761
           4       0.00      0.00      0.00       273
           5       0.24      0.01      0.02       930
           6       0.00      0.00      0.00        48
           7       0.95      1.00      0.97      1197

    accuracy                           0.76     10479
   macro avg       0.42      0.37      0.35     10479
weighted avg       0.66      0.76      0.68     10479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adabstmodel = AdaBoostClassifier(n_estimators=50, random_state=42)
adabstmodel.fit(X_train.tolist(), y_train.tolist())
val_pred2=adabstmodel.predict(X_test.tolist())
accuracy = accuracy_score(y_test, val_pred2)

print("Dev Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, val_pred2))

Dev Accuracy: 64.9012

               precision    recall  f1-score   support

           0       0.61      0.92      0.73      2311
           1       0.65      0.79      0.71      4613
           2       0.00      0.00      0.00       346
           3       0.00      0.00      0.00       761
           4       0.07      0.01      0.02       273
           5       0.15      0.05      0.07       930
           6       0.00      0.00      0.00        48
           7       0.95      0.83      0.89      1197

    accuracy                           0.65     10479
   macro avg       0.30      0.32      0.30     10479
weighted avg       0.54      0.65      0.58     10479



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
